# Import libs

In [ ]:
!nvidia-smi

Thu Aug 12 10:44:48 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8    30W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
pip install --upgrade transformers

     |████████████████████████████████| 2.6 MB 4.0 MB/s 
     |████████████████████████████████| 636 kB 26.2 MB/s 
     |████████████████████████████████| 895 kB 45.1 MB/s 
     |████████████████████████████████| 3.3 MB 42.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pip install simpletransformers

     |████████████████████████████████| 221 kB 4.1 MB/s 
     |████████████████████████████████| 124 kB 47.6 MB/s 
     |████████████████████████████████| 76 kB 4.8 MB/s 
     |████████████████████████████████| 8.0 MB 49.6 MB/s 
     |████████████████████████████████| 43 kB 2.0 MB/s 
     |████████████████████████████████| 264 kB 44.2 MB/s 
     |████████████████████████████████| 1.6 MB 38.9 MB/s 
     |████████████████████████████████| 1.2 MB 50.1 MB/s 
     |████████████████████████████████| 170 kB 48.2 MB/s 
     |████████████████████████████████| 133 kB 51.8 MB/s 
     |████████████████████████████████| 97 kB 7.0 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
     |████████████████████████████████| 243 kB 48.7 MB/s 
     |████████████████████████████████| 118 kB 48.5 MB/s 
     |████████████████████████████████| 4.2 MB 41.5 MB/s 
     |████████████████████████████████| 75 kB 2.9 MB/s 
     |████████████████████████████████| 111 kB 50.4 MB/s 
     |███████████████████

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import torch
import pandas as pd
import logging
import wandb
import os
from sklearn.model_selection import train_test_split

cuda_available = torch.cuda.is_available()

# Import data

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
os.chdir('/content/gdrive/My Drive/BERT/SA')

Mounted at /content/gdrive


In [ ]:
df = pd.read_csv('preprocessed.csv', index_col= 0)
df = df.drop('Unnamed: 0.1', axis = 1)
df.head()

,content,label,avg_length,sen_word_seg
0,Nghệ sỹ hài Dương Hoài Linh bị chỉ trích vì gi...,1,1683,Nghệ_sỹ hài Dương_Hoài_Linh bị chỉ_trích vì gi...
1,Người dân bị lập biên bản vì không muốn làm tr...,1,1588,Người_dân bị lập biên_bản vì không muốn làm tr...
2,Ngân hàng thế giới nói dòng vốn đầu tư trực ti...,1,1412,Ngân_hàng thế_giới nói dòng vốn đầu_tư trực_ti...
3,Người Việt uống nhiều bia rượu hơn trong đại d...,1,1017,Người Việt uống nhiều bia rượu hơn trong đại_d...
4,Nhiều người Việt Nam muốn đi Hoa Kỳ tiêm vacci...,1,1402,Nhiều người Việt_Nam muốn đi Hoa_Kỳ tiêm vacci...


In [ ]:
df = df[['sen_word_seg', 'label']]
df['sen_word_seg'] = df['sen_word_seg'].astype('category')
df = df.rename(columns = {'sen_word_seg': 'text', 'label': 'labels'})
df

,text,labels
0,Nghệ_sỹ hài Dương_Hoài_Linh bị chỉ_trích vì gi...,1
1,Người_dân bị lập biên_bản vì không muốn làm tr...,1
2,Ngân_hàng thế_giới nói dòng vốn đầu_tư trực_ti...,1
3,Người Việt uống nhiều bia rượu hơn trong đại_d...,1
4,Nhiều người Việt_Nam muốn đi Hoa_Kỳ tiêm vacci...,1
...,...,...
10179,Thủ_tướng Nguyễn_Xuân_Phúc dự Hội_nghị tổng_kế...,0
10180,Chủ_tịch Quốc_hội Nguyễn_Thị_Kim_Ngân chủ_trì ...,0
10181,Chủ_tịch nước Trần_Đại_Quang dự chương_trình n...,0
10182,Ông Hầu_A_Lềnh giữ chức Phó chủ_tịch - Tổng_th...,0


# Split data

In [ ]:
df['labels'].value_counts()

0    30451
1    19066
Name: labels, dtype: int64

In [ ]:
df_1 = df[df['labels'] == 1]
df_0 = df[df['labels'] == 0]

# df_1 = df_1.sample(frac=1).reset_index(drop=True)
# df_0 = df_0.sample(frac=1).reset_index(drop=True)

test_df = pd.concat([df_0.iloc[-5000:, :], df_1.iloc[-5000:, :]])
test_df = test_df.sample(frac=1, random_state = 5).reset_index(drop=True)

train_subset_df = pd.concat([df_0.iloc[:-9000, :], df_1.iloc[:-5000, :]])
train_subset_df = train_subset_df.sample(frac=1, random_state = 5).reset_index(drop=True)

In [ ]:
print(train_subset_df['labels'].value_counts())
print(test_df['labels'].value_counts())
print(train_subset_df.shape)
#0.6, 0.4

0    21451
1    14066
Name: labels, dtype: int64
1    5000
0    5000
Name: labels, dtype: int64
(35517, 2)


In [ ]:
w_1 = 35517 / (2 * 12066)
w_0 = 35517 / (2 * 21451)
w_0

0.8278635028669992

# Sweep hyperparameters

## Define model

In [ ]:
# model = ClassificationModel( "xlmroberta", "microsoft/Multilingual-MiniLM-L12-H384", 
#                                 args= create_args_model_2(), 
#                                 use_cuda = cuda_available, 
#                                 num_labels = 2,  
#                                 weight = [1.5, 0.8])

# for para in model.get_named_parameters():
#     print(para)

In [ ]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# MODEL 1 ARGS: tuning all classifier --------------------------
model_args = {
    'num_train_epochs': 2,
    # 'save_eval_checkpoints': False,
    'train_batch_size': 40,
    # 'eval_batch_size': 256,
    'overwrite_output_dir': True,
    # 'sliding_window': True,
    'use_early_stopping': True,
    'no_cache': True,
    'max_seq_length': 300,
    'wandb_project': 'nlp_bert cross validation (early stopping)',
    'early_stopping_consider_epochs': True,
    'save_steps': -1,  # don't save model every number of steps


    # only cross validation, try not to save on disk-----------------
    'save_model_every_epoch': False,
    'save_optimizer_and_scheduler': False,
    'save_eval_checkpoints': False,
    'no_save': True,

    # test early stopping
    'evaluate_during_training': True,
    'early_stopping_delta': 0.01,
    'evaluate_during_training_steps': 1000,
    'evaluate_during_training_verbose': True,
    'early_stopping_patience': 5,
}

# MODEL 2 ARGS: only tuning in classifier layer---------------------------------------
    # increase adam epsilon to slower the optimizer
    # 'adam_epsilon' : 1e-6

def create_args_model_2():
    model_args = ClassificationArgs()

    # Set custom weight bias to train only
    model_args.custom_parameter_groups = [
        {
            "params": ["classifier.dense.weight", 
                       "classifier.dense.bias"],
            "lr": 1e-3,
        }
    ]
    model_args.train_custom_parameters_only = True
    

    model_args.evaluate_during_training = True 
    model_args.num_train_epochs = 3
    model_args.train_batch_size: 40
    model_args.overwrite_output_dir = True
    model_args.use_early_stopping = True
    model_args.no_cache = True
    model_args.max_seq_length = 300
    model_args.wandb_project = 'nlp_bert cross validation (only last layer tuning)'
    model_args.early_stopping_consider_epochs = True
    model_args.save_steps = -1  # don't save model every number of steps


    # only cross validation, try not to save on disk-----------------
    model_args.save_model_every_epoch = False
    model_args.save_optimizer_and_scheduler = False
    model_args.save_eval_checkpoints = False
    model_args.no_save = True

    # test early stopping
    # model_args.early_stopping_delta = 0.01    # %decrease in loss function
    # model_args.early_stopping_patience = 5  # % wait 5 evaluation < delta to stop 
    model_args.evaluate_during_training_steps = 2000   #takes 1000 steps then evaluate
    
    # model_args.early_stopping_metric = "f1"
    # model_args.early_stopping_metric_minimize = False

    return model_args

# MODEL 3 ARGS: only tuning in layers 0, 10, 11, classifier---------------------------------------
def create_args_model_2():
    model_args = ClassificationArgs()

    model_args.wandb_project = 'sweep v4'

    # Set custom weight bias to train only
    model_args.custom_parameter_groups = [
        {
            "params": ["classifier.dense.weight", 
                       "classifier.dense.bias"],
            # "lr": 1e-5
        }
    ]

    model_args.custom_layer_parameters = [
        {
            "layer": 10,
            # "lr": 1e-5,
        },

        {
            "layer": 11,
            # "lr": 1e-5,
        },

        {
            "layer": 0,
            # "lr": 1e-5,
        },
    ]

    model_args.train_custom_parameters_only = True

    model_args.evaluate_during_training = True 
    model_args.train_batch_size: 40
    model_args.overwrite_output_dir = True
    model_args.use_early_stopping = True
    model_args.no_cache = True
    model_args.max_seq_length = 300
    model_args.early_stopping_consider_epochs = True
    model_args.save_steps = -1  # don't save model every number of steps
    model_args.num_train_epochs = 3

    # only cross validation, try not to save on disk-----------------
    model_args.save_model_every_epoch = False
    model_args.save_optimizer_and_scheduler = False
    model_args.save_eval_checkpoints = False
    model_args.no_save = True

    # test early stopping
    # model_args.early_stopping_delta = 5    # %decrease in loss function
    # model_args.early_stopping_patience = 2  # % wait 5 evaluation < delta to stop 
    model_args.evaluate_during_training_steps = 2000   #takes 1000 steps then evaluate
    
    # model_args.early_stopping_metric = "f1"
    # model_args.early_stopping_metric_minimize = False

    return model_args

## Sweep config

In [ ]:
# def hyper_sweep(train_subset_df, train_index, text_index):
#     # Dividing data into folds

#     train_temp_df = train_subset_df.iloc[train_index]
#     eval_temp_df = train_subset_df.iloc[test_index]

#     # Sweep config here
#     sweep_config = {
#         "method": "bayes",  
#         "metric": {"name": "f1", "goal": "maximize"},
#         "parameters": {
#             "num_train_epochs": {"values": [2]},
#             "param_classifier.dense.weight": {"min": 1e-6, "max": 1e-2},
#             "param_classifier.dense.bias": {"min": 1e-6, "max": 1e-2},
#             "layer_0": {"min": 1e-6, "max": 1e-2}, 
#             "layer_10": {"min": 1e-6, "max": 1e-2}, 
#             "layer_11": {"min": 1e-6, "max": 1e-2}, 
#             "weight_decay": {"values": [1e-2, 0e-1, 1, 10]}
#             },
#             "early_terminate": {"type": "hyperband", "min_iter": 2,},
#     }

#     sweep_id = wandb.sweep(sweep_config, project="nlp_bert hyper sweep (layer 0 - 10 - 11 - classifier layers tuning)")
#     id = wandb.util.generate_id() # Id for resuming
    
#     def train():



#         # Create a ClassificationModel
#         cprint((f'\n======= Initialize model ========\n'), 'green')
#         wandb.init(id = id, resume = True) # resume allowed

#         # Model 1===========
#         # model = ClassificationModel( "xlmroberta", "microsoft/Multilingual-MiniLM-L12-H384", 
#         #                             args=model_args, 
#         #                             use_cuda = cuda_available, 
#         #                             num_labels = 2,  
#         #                             weight = [0.8, 1.5])
#         # =================

#         # Augment train_temp_df using translation

#         # Model 2==========
#         model = ClassificationModel( "xlmroberta", "microsoft/Multilingual-MiniLM-L12-H384", 
#                                     args= create_args_model_2(), 
#                                     use_cuda = cuda_available, 
#                                     num_labels = 2,  
#                                     weight = [1.5, 0.8])
        
#         #==================


#         # Train model
#         cprint(f'\n======= Train model ========\n', 'green')
#         model.train_model(train_temp_df, eval_df = eval_temp_df, f1 = sklearn.metrics.f1_score)

#         # # Evaluate the model
#         # cprint(f'\n======= Evaluate model ========\n', 'green')
#         # result, model_outputs, wrong_predictions = model.eval_model(eval_temp_df, 
#         #                         f1 = sklearn.metrics.f1_score, 
#         #                         acc = sklearn.metrics.accuracy_score, 
#         #                         conf = sklearn.metrics.confusion_matrix, 
#         #                         report = sklearn.metrics.classification_report)
        
#         # print(result['conf'])
#         # print(result['report'])

#         # wandb.log(result)

#         # MODEL 2-----
#         wandb.join()

#     # Sweep id here "boswzbvt"
#     wandb.agent("boswzbvt", project="nlp_bert hyper sweep (layer 0 - 10 - 11 - classifier layers tuning)", function=train)
#     # -----------

#     # Free gpu memory
#     del model
#     del train_temp_df
#     del eval_temp_df
#     del model_outputs 
#     del wrong_predictions

#     torch.cuda.empty_cache()
#     gc.collect()



#     return result


def hyper_sweep(train_subset_df):
    # Dividing data into folds

    l = len(train_subset_df)
    train_temp_df = train_subset_df.iloc[: ( 2 * l//3)]
    eval_temp_df = train_subset_df.iloc[(l//3):]

    # Sweep config here
    sweep_config = {
        "method": "bayes",  
        "metric": {"name": "eval_loss", "goal": "minimize"},
        "parameters": {
            "param_classifier.dense.weight": {"min": 1e-8, "max": 1e-4},
            "param_classifier.dense.bias": {"min": 1e-8, "max": 1e-4},
            "layer_0": {"min": 1e-8, "max": 1e-4}, 
            "layer_10": {"min": 1e-8, "max": 1e-4}, 
            "layer_11": {"min": 1e-8, "max": 1e-4}, 
            "weight_decay": {"values": [1e-3, 1e-2, 1e-1, 0]}
            },
            "early_terminate": {"type": "hyperband", "min_iter": 2,},
    }

    # Initialize sweep
    sweep_id = wandb.sweep(sweep_config, project="sweep v4")

    # Resume existed sweep container
    sweep_id = 'dlep0q69'
    
    def train():



        # Create a ClassificationModel
        cprint((f'\n======= Initialize model ========\n'), 'green')
        wandb.init(id = sweep_id, resume = True) # resume allowed

        # Model 1===========
        # model = ClassificationModel( "xlmroberta", "microsoft/Multilingual-MiniLM-L12-H384", 
        #                             args=model_args, 
        #                             use_cuda = cuda_available, 
        #                             num_labels = 2,  
        #                             weight = [0.8, 1.5])
        # =================

        # Augment train_temp_df using translation

        # Model 2==========
        model = ClassificationModel( "xlmroberta", "microsoft/Multilingual-MiniLM-L12-H384", 
                                    args= create_args_model_2(), 
                                    use_cuda = cuda_available, 
                                    num_labels = 2)
        
        #==================


        # Train model
        cprint(f'\n======= Train model ========\n', 'green')
        model.train_model(train_temp_df, eval_df = eval_temp_df, f1 = sklearn.metrics.f1_score)

        # # Evaluate the model
        # cprint(f'\n======= Evaluate model ========\n', 'green')
        # result, model_outputs, wrong_predictions = model.eval_model(eval_temp_df, 
        #                         f1 = sklearn.metrics.f1_score, 
        #                         acc = sklearn.metrics.accuracy_score, 
        #                         conf = sklearn.metrics.confusion_matrix, 
        #                         report = sklearn.metrics.classification_report)
        
        # print(result['conf'])
        # print(result['report'])

        # wandb.log(result)

        # MODEL 2-----
        wandb.join()

    # Sweep id here "boswzbvt"
    wandb.agent(sweep_id, project="sweep v4", function=train)
    # -----------

    # Free gpu memory
    del model
    del train_temp_df
    del eval_temp_df
    del model_outputs 
    del wrong_predictions

    torch.cuda.empty_cache()
    gc.collect()



    return result

## Running

In [ ]:
from sklearn.model_selection import StratifiedKFold, train_test_split
import sklearn
from tqdm.notebook import tqdm
from termcolor import colored, cprint
import wandb
import gc

k_folds = 4
kfold = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state = 5)  # fix random shuffle



cv_results = []

# TEST 
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
# TEST

# for fold, (train_index, test_index) in enumerate(kfold.split(train_subset_df['text'], train_subset_df['labels'])):
#     cprint((f'\n================ Fold: {fold} =================\n\n'), 'white', 'on_green')

#     if fold == 0:
#         res = hyper_sweep(train_subset_df, train_index, test_index)
#         cv_results.append(res)




# for fold, (train_index, test_index) in enumerate(kfold.split(train_subset_df['text'], train_subset_df['labels'])):
#     cprint((f'\n================ Fold: {fold} =================\n\n'), 'white', 'on_green')

#     if fold == 0:
res = hyper_sweep(train_subset_df)

    

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
INFO:wandb.agents.pyagent:Starting sweep agent: entity=None, project=sweep v4, count=None


Create sweep with ID: z4h22j5m
Sweep URL: https://wandb.ai/tuanle/sweep%20v4/sweeps/z4h22j5m


...

Running Epoch 2 of 3:   0%|          | 0/2960 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/23678 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/23678 [00:00<?, ?it/s]